In [2]:
import pandas as pd
import numpy as np
import os, re, glob
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from IPython.display import IFrame, Image
import warnings


In [5]:
root = 'Covid22'
previous_date = '02-01-2022'
recent_date = '03-31-2022'
duplicate_columns = {'Lat':'Latitude', 'Long_': 'Longitude', 'Incidence_Rate':'Incident_Rate', 'Case-Fatality_Ratio':'Case_Fatality_Ratio', 'Province/State':'Province_State'}

recent_df = pd.read_csv(os.path.join(root,(recent_date+'.csv')))
previous_df = pd.read_csv(os.path.join(root,(previous_date+'.csv')))

for key, value in duplicate_columns.items():
    if key in recent_df.columns:
        recent_df = recent_df.rename(columns={key: value})
if key in previous_df.columns:
    previous_df = previous_df.rename(columns={key: value})

FileNotFoundError: [Errno 2] No such file or directory: 'Covid22/03-31-2022.csv'

In [6]:
recent_df.head()

NameError: name 'recent_df' is not defined

In [ ]:
previous_df.head()

In [7]:
current_df = pd.DataFrame(columns=['Province_State','Country_Region','Confirmed','Deaths'])
current_df['Province_State'] = recent_df['Province_State']
current_df['Country_Region'] = recent_df['Country_Region']
current_df['Confirmed'] = recent_df['Confirmed'] - previous_df['Confirmed']
current_df['Deaths'] = recent_df['Deaths'] - previous_df['Deaths']

NameError: name 'recent_df' is not defined

In [8]:
current_df.head()

,Province_State,Country_Region,Confirmed,Deaths


In [9]:
current_df.shape

(0, 4)

In [10]:
name_number = 'MameFasseSALL-2441202_4.csv'
current_df.to_csv(name_number, index=False)

In [11]:
data = pd.read_csv(name_number)

In [12]:
data.head()

,Province_State,Country_Region,Confirmed,Deaths


In [13]:
name_number

'MameFasseSALL-2441202_4.csv'

In [14]:
data.shape

(0, 4)

In [15]:
data.count()

Province_State    0
Country_Region    0
Confirmed         0
Deaths            0
dtype: int64

In [16]:
#Q1
data.isnull().sum()

Province_State    0
Country_Region    0
Confirmed         0
Deaths            0
dtype: int64

In [17]:
data.loc[data['Province_State'].isnull(),'Province_State']= data['Country_Region']

In [18]:
data.head()

,Province_State,Country_Region,Confirmed,Deaths


In [19]:
states = data['Province_State'].unique()
print('Number of Unique Sataes: ', len(states))

Number of Unique Sataes:  0


In [20]:
#Q@
country = data['Country_Region'].unique()
print('Number of Country: ', len(country))

Number of Country:  0


In [ ]:
pip install geopy

In [ ]:
import time
from geopy.geocoders import Nominatim
from tqdm import tqdm
def get_lat_lon(place, user_agent):
    geolocator = Nominatim(user_agent=user_agent)
    location = geolocator.geocode(place)
    if location:
       lat_lon = location.latitude, location.longitude
       return [float(i) for i in lat_lon]
    return [None, None]
user_agent = name_number
geo_lat = []
geo_lon = []
not_found = []
for state in tqdm(states):
     time.sleep(0.5) # Adding delay to avoid rate limiting
     lat_lon = [None, None]
    try:
        lat_lon = get_lat_lon(state, user_agent)
        
    except:
        not_found.append(state)
    geo_lat.append(lat_lon[0])
    geo_lon.append(lat_lon[1])
    
if len(not_found) > 0:
    print("Locations are not found for:", not_found)
else:
    print("Found all the locations")

In [ ]:
states_list = states.tolist()
lats = []
lons = []
for i, r in data.iterrows():
state = r['Province_State']
index_list = states_list.index(state)
lats.append(geo_lat[index_list])
lons.append(geo_lon[index_list])
data['Latitude'] = lats
data['Longitude'] = lons

In [ ]:
data.head()

In [ ]:
data['differnce_Longitude'] = data['Longitude'] - recent_df['Longitude']
data['difference_Latitude'] = data['Latitude'] - recent_df['Latitude']

In [ ]:
data.head()

In [ ]:
data = data[data['Latitude'].notna()]

In [ ]:
data.shape

In [ ]:
clustering_data = data[['Confirmed','Deaths']]

In [ ]:
clustering_data.head()

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit(clustering_data).transform(clustering_data.astype(np.float64))

In [ ]:
cluster_range = range(1,20)
cluster_errors = []
for num_clusters in cluster_range:
clusters = KMeans(num_clusters)
clusters.fit(X_scaled)
cluster_errors.append(clusters.inertia_)
clusters_df = pd.DataFrame({'num_clusters': cluster_range, 'cluster_errors': cluster_errors})
plt.figure(figsize=(16,6))
plt.plot(clusters_df.num_clusters, clusters_df.cluster_errors, marker ='o')

In [ ]:
kmeans = KMeans(n_clusters = 4, init = 'k-means++', random_state = 10)
y_kmeans = kmeans.fit_predict(X_scaled)
y_kmeans1 = y_kmeans + 1
cluster = list(y_kmeans1)
clustering_data['cluster'] = cluster

In [ ]:
clustering_data.head()

In [ ]:
kmeans_mean_cluster = pd.DataFrame(round(clustering_data.groupby('cluster').mean(),1))
kmeans_mean_cluster

In [ ]:
data['cluster'] = cluster
clusters = data[['Province_State', 'cluster']]
clusters.loc[clusters['cluster']==2]

In [ ]:
data['cluster'] = cluster
clusters = data[['Province_State', 'cluster']]
clusters.loc[clusters['cluster']==3]

In [ ]:
#Q4
data['cluster'] = cluster
clusters = data[['Province_State', 'cluster']]
clusters.loc[clusters['cluster']==1]

In [ ]:
data['cluster'] = cluster
clusters = data[['Province_State', 'cluster']]
clusters.loc[clusters['cluster']==4]

In [ ]:
data.head()

In [ ]:
def get_color(cluster_id):
    if cluster_id == 2:
        return 'darked'
    if cluster_id == 1:
        return 'green'
    if cluster_id == 3:
        return 'orange'
    if cluster_id == 4:
        return 'yellow'
    data['color'] = data['cluster'].apply(lambda x: get_color(x))

In [ ]:
pip install folium

In [ ]:
import folium
this_map = folium.Map(location =[data['Latitude'].mean(), data['Longitude'].mean()], zoom_start=5)
def plot_dot(point):
    folium.CircleMarker(location = [point.Latitude, point.Longitude],radius= 2, color=point.color, weight=1).add_to(this_map)
data.apply(plot_dot, axis=1)
this_map.fit_bounds(this_map.get_bounds())
this_map.save(os.path.join('covid_map.html'))